In [4]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# ----------------------------------------------------------------------
# 1.1 Standard EF
# ----------------------------------------------------------------------
def echelon_form(A):
    # Creates a copy of A and stores it in U. This ensures the original matrix A is not modified during computation.
	U = np.copy(A)
	
	m, n = A.shape
	j = 0
	pivot_columns = []

	for i in range(0,m):
		found_pivot = False
		# Loop that continues until either a pivot is found (found_pivot == True) or all columns have been checked (j >= n).
		while not found_pivot and j < n:
			# Find row with maximum absolute value in column j from row i onward
			max_row = np.argmax(abs(U[i:m, j])) + i

			# If the current entry is zero, swap with the max row
			if abs(U[i, j]) == 0:
				U[[i, max_row], :] = U[[max_row, i], :]

			# If after swapping we have a non-zero pivot, proceed
			if abs(U[i, j]) > 0:
				pivot_columns.append(j)
				# Eliminate entries below the pivot
				multipliers = U[i+1:m, j] / U[i, j]
				U[i+1:m, j+1:n] -= np.outer(multipliers, U[i, j+1:n])
				U[i+1:m, j] = 0
				j = j + 1
				found_pivot = True
			else:
				j = j + 1  # try next column
				found_pivot = False
	return (U, pivot_columns)

<details>
<summary><b>Explanation of echelon_form </b></summary>
---

### **Function Purpose**
```python
def echelon_form(A):
```
- The goal is to transform matrix `A` into **row echelon form**—a simplified upper-triangular-like structure where:
  - All nonzero rows are above any rows of all zeros.
  - The leading coefficient (pivot) of a nonzero row is always strictly to the right of the pivot in the row above.
- The function returns:
  - `U`: the echelon form of `A`.
  - `pivot_columns`: a list of column indices where pivots occur.

---

### **Step 1: Preserve Original Matrix**
```python
U = np.copy(A)
```
- Creates a **copy** of input matrix `A` to avoid modifying the original during computation.

---

### **Step 2: Initialize Dimensions and Tracking Variables**
```python
m, n = A.shape
j = 0
pivot_columns = []
```
- `m`, `n`: number of rows and columns.
- `j`: current column index being examined for a pivot (starts at 0).
- `pivot_columns`: will store the column index of each pivot found.

---

### **Step 3: Iterate Over Rows (Potential Pivot Rows)**
```python
for i in range(0, m):
```
- Loop over each row `i` from top to bottom. Each row may host **at most one pivot**.

---

### **Step 4: Search for a Pivot in Current or Later Columns**
```python
found_pivot = False
while not found_pivot and j < n:
```
- For row `i`, scan columns starting from `j` (to ensure staircase structure).
- Stop when either:
  - A valid pivot is found (`found_pivot = True`), or
  - All columns have been checked (`j >= n`).

---

### **Step 5: Partial Pivoting — Find Best Row for Stability**
```python
max_row = np.argmax(abs(U[i:m, j])) + i
```
- Looks **only from row `i` downward** in column `j`.
- Finds the row with the **largest absolute value** in that column segment.
- `np.argmax` returns an index relative to the slice `U[i:m, j]`, so we add `i` to get the actual row index.

> **Why?** This is **partial pivoting**, which improves numerical stability by avoiding division by very small numbers.

---

### **Step 6: Swap Rows if Needed**
```python
if abs(U[i, j]) == 0:
    U[[i, max_row], :] = U[[max_row, i], :]
```
- If the current entry `U[i, j]` is zero, swap row `i` with `max_row`.
- Even if `max_row == i`, swapping is harmless but unnecessary—could be optimized, but clarity is prioritized here.

> **Note**: This ensures that if *any* non-zero exists below, it moves up.

---

### **Step 7: Check if Pivot is Valid (Non-Zero)**
```python
if abs(U[i, j]) > 0:
```
- After swapping, check if the new `U[i, j]` is non-zero (within floating-point tolerance).
- If yes → proceed to eliminate entries below.

---

### **Step 8: Record Pivot Column**
```python
pivot_columns.append(j)
```
- Store column index `j` as the location of the pivot for row `i`.

---

### **Step 9: Eliminate Entries Below the Pivot**
```python
multipliers = U[i+1:m, j] / U[i, j]
U[i+1:m, j+1:n] -= np.outer(multipliers, U[i, j+1:n])
U[i+1:m, j] = 0
```
- **Compute multipliers**: how much to scale the pivot row to cancel entries below.
- **Update submatrix**: subtract scaled pivot row from rows below, but **only from column `j+1` onward**.
  - Why not column `j`? Because we’ll explicitly zero it next.
- **Explicitly zero out column `j` below pivot**:
  ```python
  U[i+1:m, j] = 0
  ```
  - This avoids tiny floating-point residuals and ensures clean zeros.

> **Use of `np.outer`**: Efficiently applies the multiplier vector to the pivot row segment, broadcasting the elimination across all affected rows and columns.

---

### **Step 10: Move to Next Column & Mark Pivot Found**
```python
j = j + 1
found_pivot = True
```
- Advance to the next column (`j += 1`) to maintain the echelon “staircase”.
- Exit the `while` loop since a pivot was placed in row `i`.

---

### **Step 11: No Pivot in This Column → Try Next Column**
```python
else:
    j = j + 1  # try next column
    found_pivot = False
```
- If even after swapping `U[i, j]` is still zero (i.e., entire column `j` from row `i` down is zero), then **no pivot in this column**.
- Increment `j` to examine the next column, but stay on the same row `i`.

> This handles **rank-deficient** matrices and zero columns gracefully.

---

### **Step 12: Return Result**
```python
return (U, pivot_columns)
```
- Returns:
  - `U`: matrix in (approximate) row echelon form.
  - `pivot_columns`: list of pivot column indices (useful for rank, basis extraction, etc.).

---

### **Key Notes & Assumptions**
- **Numerical Tolerance**: Uses `abs(x) > 0`—in practice, a small tolerance (e.g., `1e-12`) is safer for floating-point arithmetic.
- **Not Reduced Echelon Form**: This is **not** RREF—no backward elimination or pivot normalization to 1.
- **Partial Pivoting Only**: Does not consider full pivoting (column swaps), so column order is preserved.

This implementation balances clarity, correctness, and efficiency for educational or moderate-scale use.

</details>

In [6]:
# ----------------------------------------------------------------------
# 1.2 Verobose EF
# ----------------------------------------------------------------------
def echelon_form_verbose(A):
	"""Echelon form with intermediate matrix display."""
	U = np.copy(A).astype(float)
	m, n = A.shape
	j = 0
	pivot_columns = []
	
	print("Initial matrix:")
	print(U)
	print()
	
	for i in range(0, m):
		found_pivot = False
		while not found_pivot and j < n:
			max_row = np.argmax(abs(U[i:m, j])) + i
			
			if abs(U[i, j]) == 0:
				U[[i, max_row], :] = U[[max_row, i], :]
				print(f"Row swap: R{i+1} <-> R{max_row+1}")
				print(U)
				print()
			
			if abs(U[i, j]) > 1e-15:
				pivot_columns.append(j)
				print(f"Pivot found at position ({i+1}, {j+1}) = {U[i, j]:.3f}")
				
				multipliers = U[i+1:m, j] / U[i, j]
				for k, mult in enumerate(multipliers):
					if abs(mult) > 1e-15:
						print(f"R{i+k+2} = R{i+k+2} - ({mult:.3f}) * R{i+1}")
				
				U[i+1:m, j+1:n] -= np.outer(multipliers, U[i, j+1:n])
				U[i+1:m, j] = 0
				
				print("After elimination:")
				print(U)
				print()
				
				j += 1
				found_pivot = True
			else:
				j += 1
				found_pivot = False
	
	print(f"Final echelon form with pivot columns: {pivot_columns}")
	return (U, pivot_columns)

In [7]:
# ----------------------------------------------------------------------
# 2.1 Standard RREF
# ----------------------------------------------------------------------
def RREF(A):
    U = np.copy(A).astype(float)
    (m, n) = A.shape
    j = 0
    pivot_columns = []
    for i in range(0, m):
        found_pivot = False
        while not found_pivot and j < n:
            indm = np.argmax(abs(U[i:m, j]))
            indm = indm + 1
            if abs(U[indm, j]) == 0:
                U[[i, indm], :] = U[[indm, i], :]
            if abs(U[i, j]) > 0:
                pivot_columns.append(j)
                M = U[i+1:m, j] / U[i, j]
                U[i+1:m, j+1:n] = U[i+1:m, j+1:n] - np.outer(M, U[i, j+1:n])
                U[i+1:m, j] = 0
                U[i, j:n] = U[i, j:n] / U[i, j]
                if i > 0:
                    M = U[0:i, j] / U[i, j]
                    U[0:i, j:n] = U[0:i, j:n] - np.outer(M, U[i, j:n])
                found_pivot = True
                j = j + 1
            else:
                found_pivot = False
                j = j + 1
    return (U, pivot_columns)

<details>
<summary><b>📘 Click to expand: Detailed explanation of <code>rref</code> (Reduced Row Echelon Form)</b></summary>

This function computes the **Reduced Row Echelon Form (RREF)** of a matrix using Gaussian elimination with partial pivoting, forward elimination, pivot normalization, and backward elimination.

> 🔸 **RREF properties**:
> - All nonzero rows are above any rows of all zeros.
> - The leading entry (pivot) of each nonzero row is **1**.
> - Each pivot is the **only nonzero entry** in its column.
> - Pivot positions move strictly to the right as you go down the rows.

---

### **Function Signature**
```python
def rref(A):
```
- **Input**: A matrix `A`.
- **Output**: A tuple `(U, p)` where:
  - `U` is the matrix in reduced row echelon form.
  - `p` is a list of column indices where pivots occur.

---

### **Step 1: Initialize Working Copy**
```python
U = np.copy(A)
(m, n) = A.shape
j = 0
p = []
```
- `U` is a modifiable copy of the input matrix.
- `m` and `n` are the number of rows and columns.
- `j` tracks the current column under consideration.
- `p` will store the indices of pivot columns.

---

### **Step 2: Process Each Row as a Potential Pivot Row**
```python
for i in range(0, m):
    ech = 1
    while (ech == 1) and (j < n):
```
- Loop over each row `i` from top to bottom.
- The inner `while` loop scans columns starting at `j` until a pivot is found or all columns are exhausted.
- `ech` acts as a control flag (`1` = still searching for a pivot in this row).

---

### **Step 3: Partial Pivoting – Select Best Pivot Row**
```python
indm = np.argmax(abs(U[i:m, j]))
indm = indm + i
```
- Among rows `i` through `m−1`, find the one with the largest absolute value in column `j`.
- `indm` is converted to an absolute row index by adding `i`.

This improves numerical stability by choosing the most significant entry as the pivot candidate.

---

### **Step 4: Row Swap if Necessary**
```python
if abs(U[indm, j]) == 0:
    U[[i, indm], :] = U[[indm, i], :]
```
- If the best candidate in column `j` is zero, the entire segment is zero—no pivot exists in this column.
- The swap ensures that if a nonzero exists, it is moved to the current row `i`.

---

### **Step 5: Confirm Valid Pivot**
```python
if abs(U[i, j]) > 0:
```
- After potential swapping, check whether the current entry is nonzero.
- If so, proceed to eliminate and normalize.

---

### **Step 6: Record Pivot Column**
```python
p.append(j)
```
- Column `j` contains a pivot in row `i`; store its index.

---

### **Step 7: Eliminate Entries Below the Pivot**
```python
M = U[i+1:m, j] / U[i, j]
U[i+1:m, j+1:n] = U[i+1:m, j+1:n] - np.outer(M, U[i, j+1:n])
U[i+1:m, j] = 0
```
- Compute multipliers `M` to cancel entries below the pivot.
- Update the submatrix from column `j+1` onward using vectorized outer product.
- Explicitly zero out column `j` below row `i` for clarity and numerical cleanliness.

---

### **Step 8: Normalize the Pivot Row**
```python
U[i, j:n] = U[i, j:n] / U[i, j]
```
- Divide the entire pivot row (from column `j` to the end) by the pivot value.
- This ensures the pivot becomes exactly **1**, satisfying a key RREF requirement.

---

### **Step 9: Eliminate Entries Above the Pivot**
```python
if i > 0:
    M = U[0:i, j] / U[i, j]
    U[0:i, j:n] = U[0:i, j:n] - np.outer(M, U[i, j:n])
```
- For all rows above `i`, subtract appropriate multiples of the pivot row.
- Since the pivot is now `1`, `M` equals the current values in column `j` above.
- This clears the column above the pivot, ensuring it is the **only nonzero** entry in its column.

---

### **Step 10: Advance to Next Column**
```python
j = j + 1
ech = 0
```
- Move to the next column to maintain the staircase structure.
- Set `ech = 0` to exit the `while` loop and proceed to the next row.

---

### **Step 11: Skip Zero Columns**
```python
else:
    j = j + 1
    ech = 1
```
- If no pivot is found in column `j` (entire column is zero from row `i` downward), skip to the next column.
- Remain on the same row `i` to search for a pivot in later columns.

---

### **Return Result**
```python
return (U, p)
```
- `U`: the input matrix transformed into reduced row echelon form.
- `p`: list of pivot column indices, useful for determining rank, basis vectors, or solution structure.

---

### 💡 Additional Notes
- The algorithm handles **rank-deficient** and **rectangular** matrices correctly.
- Uses vectorized operations (`np.outer`) for efficient elimination.
- Maintains the original matrix unchanged by working on a copy.

This implementation provides a clear, step-by-step path to RREF suitable for educational use and moderate-scale computation.

</details>

In [8]:
# ----------------------------------------------------------------------
# 2.2 Verbose RREF
# ----------------------------------------------------------------------
def RREF_verbose(A):
    """Reduced Row Echelon Form (RREF) with detailed step-by-step output."""
    
    U = np.copy(A).astype(float)
    m, n = A.shape
    j = 0
    pivot_columns = []
    
    print("Initial matrix:")
    print(U)
    print()
    
    for i in range(0, m):
        found_pivot = False
        while not found_pivot and j < n:
            # Find the row with the largest absolute value in column j from row i onward
            max_row_offset = np.argmax(np.abs(U[i:m, j]))
            max_row = max_row_offset + i

            # If current pivot candidate is zero, swap with max_row
            if np.abs(U[i, j]) < 1e-15:
                if np.abs(U[max_row, j]) > 1e-15:
                    U[[i, max_row], :] = U[[max_row, i], :]
                    print(f"Row swap: R{i+1} <-> R{max_row+1}")
                    print(U)
                    print()
                else:
                    # Entire column is zero below row i → move to next column
                    j += 1
                    continue

            # Now U[i, j] is non-zero → valid pivot
            if np.abs(U[i, j]) > 1e-15:
                pivot_columns.append(j)
                pivot_val = U[i, j]
                print(f"Pivot found at ({i+1}, {j+1}) = {pivot_val:.6g}")

                # Step 1: Normalize pivot row so that pivot becomes 1
                U[i, j:n] = U[i, j:n] / pivot_val
                print(f"Normalize R{i+1}: divide by {pivot_val:.6g}")
                print(U)
                print()

                # Step 2: Eliminate entries BELOW the pivot
                if i + 1 < m:
                    multipliers_below = U[i+1:m, j] / U[i, j]  # U[i,j] is now 1
                    for k, mult in enumerate(multipliers_below):
                        if np.abs(mult) > 1e-15:
                            print(f"R{i+k+2} = R{i+k+2} - ({mult:.6g}) * R{i+1}")
                    U[i+1:m, j+1:n] -= np.outer(multipliers_below, U[i, j+1:n])
                    U[i+1:m, j] = 0.0

                # Step 3: Eliminate entries ABOVE the pivot (to get RREF, not just REF)
                if i > 0:
                    multipliers_above = U[0:i, j]  # since pivot is 1, multiplier = entry itself
                    for k, mult in enumerate(multipliers_above):
                        if np.abs(mult) > 1e-15:
                            print(f"R{k+1} = R{k+1} - ({mult:.6g}) * R{i+1}")
                    U[0:i, j:n] -= np.outer(multipliers_above, U[i, j:n])

                print("After elimination above and below:")
                print(U)
                print()

                found_pivot = True
                j += 1
            else:
                found_pivot = False
                j += 1

    print(f"Final Reduced Row Echelon Form (RREF)")
    print("Pivot columns:", pivot_columns)
    return (U, pivot_columns)

In [9]:
M1 = np.array([
    [2, 1, 3], 
    [4, 2, 1], 
    [6, 3, 4]
    ])
M2 = np.array([
    [1, 2, 3, 4], 
    [2, 4, 6, 8],
    [3, 6, 9, 12]
    ])
M3 = np.array([
    [1, 0, 2],
    [0, 1, 3],
    [0, 0, 0]
    ])

In [10]:
# Test verbose function
print("\n" + "="*50)
print("VERBOSE ECHELON FORM DEMONSTRATION")
print("="*50)
print("\nTesting with Matrix 1:")
echelon_form_verbose(M1)


VERBOSE ECHELON FORM DEMONSTRATION

Testing with Matrix 1:
Initial matrix:
[[2. 1. 3.]
 [4. 2. 1.]
 [6. 3. 4.]]

Pivot found at position (1, 1) = 2.000
R2 = R2 - (2.000) * R1
R3 = R3 - (3.000) * R1
After elimination:
[[ 2.  1.  3.]
 [ 0.  0. -5.]
 [ 0.  0. -5.]]

Row swap: R2 <-> R2
[[ 2.  1.  3.]
 [ 0.  0. -5.]
 [ 0.  0. -5.]]

Pivot found at position (2, 3) = -5.000
R3 = R3 - (1.000) * R2
After elimination:
[[ 2.  1.  3.]
 [ 0.  0. -5.]
 [ 0.  0.  0.]]

Final echelon form with pivot columns: [0, 2]


(array([[ 2.,  1.,  3.],
        [ 0.,  0., -5.],
        [ 0.,  0.,  0.]]),
 [0, 2])